# 8 — Deep Learning in ROS2

> Run object detection (YOLO / MobileNet) on live camera streams inside ROS2 nodes.

**Goal:** Build a ROS2 node that receives camera images, runs a deep learning model, and publishes detection results — ready for downstream decision-making.

In [ ]:
#| default_exp deep_learning

## 8.1 Why Deep Learning in Robotics?

Traditional CV (notebook 07) works well for **simple, controlled** environments:
- Track a red ball on a white floor ✅
- Detect edges of a conveyor belt ✅

But for **complex, real-world** perception you need deep learning:
- "What objects are in front of me?" → **Object Detection** (YOLO, SSD)
- "Where can I drive?" → **Semantic Segmentation** (DeepLab, SegFormer)
- "Where is the person's hand?" → **Pose Estimation** (MediaPipe, OpenPose)
- "What is this thing?" → **Classification** (ResNet, EfficientNet)

### Object Detection in ROS2 — The Pattern

```
/camera/image_raw  →  [DL Detector Node]  →  /detections (custom msg)
                                           →  /camera/annotated (Image)
```

## 8.2 Model Choices

| Model | Speed | Accuracy | Best For |
|-------|-------|----------|----------|
| **YOLOv8n** (nano) | ⚡⚡⚡ | ⭐⭐ | Real-time on CPU |
| **YOLOv8s** (small) | ⚡⚡ | ⭐⭐⭐ | Good balance CPU/GPU |
| **YOLOv8m/l/x** | ⚡ | ⭐⭐⭐⭐ | GPU only, high accuracy |
| **MobileNet-SSD** | ⚡⚡⚡ | ⭐⭐ | Edge devices (Jetson, Pi) |
| **RT-DETR** | ⚡⚡ | ⭐⭐⭐⭐ | Transformer-based, GPU |

> **We'll use YOLOv8n** — it runs at 15-30 FPS on a CPU and detects 80 COCO classes (person, car, dog, bottle…).

### Install Ultralytics (YOLO)

```bash
pip3 install ultralytics
```

The first time you use a model, `ultralytics` auto-downloads the weights (~6 MB for yolov8n).

## 8.3 Quick YOLO Test (Outside ROS2)

Before integrating into ROS2, verify YOLO works:

In [ ]:
#| export

YOLO_TEST = '''
# Quick test — run this in Python to verify ultralytics works
from ultralytics import YOLO
import cv2

# Load YOLOv8 nano (auto-downloads on first run)
model = YOLO('yolov8n.pt')

# Run inference on a test image (or use a webcam frame)
results = model('https://ultralytics.com/images/bus.jpg')

# Print detections
for result in results:
    for box in result.boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        label = result.names[cls_id]
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        print(f'{label}: {conf:.2f} at [{x1:.0f}, {y1:.0f}, {x2:.0f}, {y2:.0f}]')

# Output:
# person: 0.88 at [17, 231, 801, 768]
# bus: 0.87 at [10, 130, 810, 720]
# ...
'''

print(YOLO_TEST)

## 8.4 The YOLO Detection Node

In [ ]:
#| export

YOLO_DETECTOR_NODE = '''
#!/usr/bin/env python3
"""ROS2 node that runs YOLOv8 object detection on camera images."""

import rclpy
from rclpy.node import Node
from rclpy.qos import qos_profile_sensor_data
from sensor_msgs.msg import Image
from std_msgs.msg import String
from cv_bridge import CvBridge
from ultralytics import YOLO
import cv2
import json
import time


class YoloDetectorNode(Node):
    def __init__(self):
        super().__init__('yolo_detector_node')

        # Parameters
        self.declare_parameter('model_path', 'yolov8n.pt')
        self.declare_parameter('confidence_threshold', 0.5)
        self.declare_parameter('device', 'cpu')        # 'cpu' or 'cuda:0'
        self.declare_parameter('input_topic', '/camera/image_raw')
        self.declare_parameter('publish_annotated', True)
        self.declare_parameter('max_fps', 15.0)        # throttle inference

        # Load the model
        model_path = self.get_parameter('model_path').value
        device = self.get_parameter('device').value
        self.get_logger().info(f'Loading model: {model_path} on {device}')

        self.model = YOLO(model_path)
        # Warm up the model
        import numpy as np
        dummy = np.zeros((480, 640, 3), dtype=np.uint8)
        self.model.predict(dummy, device=device, verbose=False)
        self.get_logger().info('Model loaded and warmed up')

        self.bridge = CvBridge()
        self.last_inference_time = 0.0

        # Subscribe to camera
        input_topic = self.get_parameter('input_topic').value
        self.subscription = self.create_subscription(
            Image, input_topic,
            self.image_callback, qos_profile_sensor_data
        )

        # Publishers
        self.annotated_pub = self.create_publisher(
            Image, '/detections/image', 10
        )
        self.detections_pub = self.create_publisher(
            String, '/detections/json', 10
        )

        self.get_logger().info(
            f'YoloDetectorNode ready — subscribed to {input_topic}'
        )

    def image_callback(self, msg: Image):
        # Throttle inference to max_fps
        now = time.time()
        min_interval = 1.0 / self.get_parameter('max_fps').value
        if (now - self.last_inference_time) < min_interval:
            return
        self.last_inference_time = now

        # Convert ROS Image → OpenCV
        cv_image = self.bridge.imgmsg_to_cv2(msg, desired_encoding='bgr8')

        # Run inference
        conf = self.get_parameter('confidence_threshold').value
        device = self.get_parameter('device').value
        results = self.model.predict(
            cv_image,
            conf=conf,
            device=device,
            verbose=False
        )

        # Process results
        detections = []
        result = results[0]

        for box in result.boxes:
            cls_id = int(box.cls[0])
            confidence = float(box.conf[0])
            label = result.names[cls_id]
            x1, y1, x2, y2 = [int(v) for v in box.xyxy[0].tolist()]

            detections.append({
                'label': label,
                'confidence': round(confidence, 3),
                'bbox': [x1, y1, x2, y2]
            })

            # Draw on image
            color = (0, 255, 0)
            cv2.rectangle(cv_image, (x1, y1), (x2, y2), color, 2)
            text = f'{label} {confidence:.2f}'
            cv2.putText(
                cv_image, text, (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2
            )

        # Publish detections as JSON
        det_msg = String()
        det_msg.data = json.dumps({
            'timestamp': msg.header.stamp.sec + msg.header.stamp.nanosec * 1e-9,
            'count': len(detections),
            'detections': detections
        })
        self.detections_pub.publish(det_msg)

        # Add FPS text
        inference_time = time.time() - now
        fps = 1.0 / max(inference_time, 1e-6)
        cv2.putText(
            cv_image, f'FPS: {fps:.1f} | Objects: {len(detections)}',
            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2
        )

        # Publish annotated image
        if self.get_parameter('publish_annotated').value:
            out_msg = self.bridge.cv2_to_imgmsg(cv_image, encoding='bgr8')
            out_msg.header = msg.header
            self.annotated_pub.publish(out_msg)

        if detections:
            summary = ', '.join(
                f'{d["label"]}({d["confidence"]:.2f})' for d in detections[:5]
            )
            self.get_logger().info(f'Detected {len(detections)}: {summary}')


def main(args=None):
    rclpy.init(args=args)
    node = YoloDetectorNode()
    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        pass
    finally:
        node.destroy_node()
        rclpy.shutdown()


if __name__ == '__main__':
    main()
'''

print(YOLO_DETECTOR_NODE)

## 8.5 Architecture of the Detection Node

```
                    ┌─────────────────────────────┐
/camera/image_raw ─►│    yolo_detector_node        │─► /detections/image  (annotated)
                    │                              │─► /detections/json   (structured data)
                    │  1. cv_bridge → numpy         │
                    │  2. YOLO inference            │
                    │  3. Draw bounding boxes       │
                    │  4. Serialize detections      │
                    │  5. Publish                   │
                    └─────────────────────────────┘
```

### Why Two Output Topics?

| Topic | Purpose | Consumer |
|-------|---------|----------|
| `/detections/image` | Visual debugging in RViz2/rqt | Human |
| `/detections/json` | Structured data for downstream nodes | Planner, logger |

### Design Decisions Explained

1. **FPS Throttling** — YOLO can't keep up with 30 FPS on CPU. We skip frames instead of building an ever-growing queue.
2. **JSON on String** — In production you'd define a custom `Detection.msg`. For learning, JSON on `std_msgs/String` is simpler.
3. **Warm-up** — The first inference is always slow (model loading, JIT). We do a dummy inference at startup.

## 8.6 Running the Detection Pipeline

```bash
# Terminal 1: Camera driver
ros2 run usb_cam usb_cam_node_exe

# Terminal 2: YOLO detector
ros2 run robot_vision yolo_detector_node --ros-args \
  -p model_path:=yolov8n.pt \
  -p confidence_threshold:=0.5 \
  -p device:=cpu \
  -p max_fps:=10.0

# Terminal 3: View annotated output
ros2 run rqt_image_view rqt_image_view
# Select /detections/image from the dropdown

# Terminal 4: Monitor detections
ros2 topic echo /detections/json
```

## 8.7 Alternative: MobileNet-SSD with OpenCV DNN

If you don't want the `ultralytics` dependency, OpenCV has a built-in DNN module:

In [ ]:
#| export

MOBILENET_NODE = '''
#!/usr/bin/env python3
"""ROS2 node with MobileNet-SSD using OpenCV DNN (no extra dependencies)."""

import rclpy
from rclpy.node import Node
from rclpy.qos import qos_profile_sensor_data
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2
import numpy as np

# COCO class names for MobileNet-SSD
CLASSES = [
    "background", "aeroplane", "bicycle", "bird", "boat",
    "bottle", "bus", "car", "cat", "chair",
    "cow", "diningtable", "dog", "horse", "motorbike",
    "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"
]


class MobileNetDetectorNode(Node):
    def __init__(self):
        super().__init__('mobilenet_detector_node')

        self.declare_parameter('prototxt_path',
            'MobileNetSSD_deploy.prototxt')
        self.declare_parameter('model_path',
            'MobileNetSSD_deploy.caffemodel')
        self.declare_parameter('confidence_threshold', 0.5)

        prototxt = self.get_parameter('prototxt_path').value
        model = self.get_parameter('model_path').value

        self.get_logger().info(f'Loading MobileNet-SSD from {model}')
        self.net = cv2.dnn.readNetFromCaffe(prototxt, model)
        self.get_logger().info('Model loaded')

        self.bridge = CvBridge()

        self.subscription = self.create_subscription(
            Image, '/camera/image_raw',
            self.image_callback, qos_profile_sensor_data
        )
        self.publisher_ = self.create_publisher(
            Image, '/detections/image', 10
        )

    def image_callback(self, msg: Image):
        cv_image = self.bridge.imgmsg_to_cv2(msg, desired_encoding='bgr8')
        h, w = cv_image.shape[:2]

        # Create a blob (preprocessed input)
        blob = cv2.dnn.blobFromImage(
            cv2.resize(cv_image, (300, 300)),
            0.007843, (300, 300), 127.5
        )

        # Run forward pass
        self.net.setInput(blob)
        detections = self.net.forward()

        conf_threshold = self.get_parameter('confidence_threshold').value

        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > conf_threshold:
                class_id = int(detections[0, 0, i, 1])
                label = CLASSES[class_id] if class_id < len(CLASSES) else '???'

                # Scale bounding box to image size
                x1 = int(detections[0, 0, i, 3] * w)
                y1 = int(detections[0, 0, i, 4] * h)
                x2 = int(detections[0, 0, i, 5] * w)
                y2 = int(detections[0, 0, i, 6] * h)

                cv2.rectangle(cv_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                text = f'{label}: {confidence:.2f}'
                cv2.putText(
                    cv_image, text, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2
                )

        out_msg = self.bridge.cv2_to_imgmsg(cv_image, encoding='bgr8')
        out_msg.header = msg.header
        self.publisher_.publish(out_msg)


def main(args=None):
    rclpy.init(args=args)
    node = MobileNetDetectorNode()
    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        pass
    finally:
        node.destroy_node()
        rclpy.shutdown()


if __name__ == '__main__':
    main()
'''

print(MOBILENET_NODE)

### YOLO vs MobileNet-SSD

| Aspect | YOLOv8 (ultralytics) | MobileNet-SSD (OpenCV DNN) |
|--------|---------------------|---------------------------|
| Classes | 80 (COCO) | 21 (VOC) |
| Install | `pip install ultralytics` | Nothing — OpenCV built-in |
| Speed (CPU) | ~50ms (nano) | ~30ms |
| Accuracy | Higher | Lower |
| Model size | 6 MB (nano) | 23 MB |
| Best for | General use | Minimal dependencies / edge |

## 8.8 GPU Acceleration

For real-time performance (30+ FPS), use a GPU:

### NVIDIA GPU with CUDA

```bash
# Install PyTorch with CUDA (https://pytorch.org/get-started)
pip3 install torch torchvision --index-url https://download.pytorch.org/whl/cu118

# Then run the node with GPU:
ros2 run robot_vision yolo_detector_node --ros-args -p device:=cuda:0
```

### NVIDIA Jetson (Edge AI)

On Jetson Nano/Xavier/Orin:
- Use **TensorRT** for maximum speed
- Export YOLO to TensorRT: `yolo export model=yolov8n.pt format=engine`
- Then load the `.engine` file instead of `.pt`

### Performance Comparison

| Platform | Model | FPS |
|----------|-------|-----|
| CPU (i7) | YOLOv8n | ~15-20 |
| GTX 1060 | YOLOv8n | ~80-100 |
| RTX 3080 | YOLOv8n | ~200+ |
| Jetson Nano | YOLOv8n (TensorRT) | ~15-20 |
| Jetson Orin | YOLOv8n (TensorRT) | ~100+ |

## 8.9 Custom Detection Messages (Production Approach)

For production, define a proper message type instead of JSON:

```bash
# Create a messages package
cd ~/ros2_ws/src
ros2 pkg create --build-type ament_cmake robot_vision_msgs
mkdir -p robot_vision_msgs/msg
```

```
# robot_vision_msgs/msg/Detection.msg
string label
float32 confidence
int32 x1
int32 y1
int32 x2
int32 y2
```

```
# robot_vision_msgs/msg/DetectionArray.msg
std_msgs/Header header
Detection[] detections
```

Custom messages are:
- **Type-safe** — catches errors at compile time
- **Efficient** — binary serialization, not JSON text
- **Discoverable** — `ros2 interface show` works on them
- **Standard** — other ROS2 tools understand them

## 8.10 Full Pipeline Launch File

In [ ]:
#| export

DL_PIPELINE_LAUNCH = '''
#!/usr/bin/env python3
"""Launch full detection pipeline: camera → YOLO → visualization."""

from launch import LaunchDescription
from launch.actions import DeclareLaunchArgument
from launch.substitutions import LaunchConfiguration
from launch_ros.actions import Node


def generate_launch_description():
    return LaunchDescription([
        # Arguments
        DeclareLaunchArgument('model', default_value='yolov8n.pt'),
        DeclareLaunchArgument('device', default_value='cpu'),
        DeclareLaunchArgument('confidence', default_value='0.5'),
        DeclareLaunchArgument('video_device', default_value='/dev/video0'),

        # Camera driver
        Node(
            package='usb_cam',
            executable='usb_cam_node_exe',
            name='camera',
            parameters=[{
                'video_device': LaunchConfiguration('video_device'),
                'image_width': 640,
                'image_height': 480,
                'framerate': 30.0,
            }],
            remappings=[('image_raw', '/camera/image_raw')]
        ),

        # YOLO detector
        Node(
            package='robot_vision',
            executable='yolo_detector_node',
            name='yolo_detector',
            parameters=[{
                'model_path': LaunchConfiguration('model'),
                'device': LaunchConfiguration('device'),
                'confidence_threshold': LaunchConfiguration('confidence'),
                'max_fps': 15.0,
            }]
        ),
    ])
'''

print(DL_PIPELINE_LAUNCH)

### Usage

```bash
# With defaults (CPU, yolov8n)
ros2 launch robot_vision detection_pipeline_launch.py

# With GPU and larger model
ros2 launch robot_vision detection_pipeline_launch.py \
  model:=yolov8s.pt device:=cuda:0 confidence:=0.3
```

## 8.11 Beyond Object Detection

The same node pattern works for other DL tasks:

| Task | Model | Output |
|------|-------|--------|
| **Segmentation** | YOLOv8-seg, DeepLabV3 | Pixel-wise class masks |
| **Pose Estimation** | YOLOv8-pose, MediaPipe | Keypoint coordinates |
| **Depth Estimation** | MiDaS, Depth Anything | Depth map per pixel |
| **OCR** | EasyOCR, PaddleOCR | Text strings |
| **Face Detection** | MediaPipe, RetinaFace | Face bounding boxes |

The architecture is always:

```
Subscribe Image → Convert → Inference → Publish Results
```

## 8.12 Summary — What You've Built

Over these 9 notebooks, you've learned:

| Notebook | Key Skill |
|----------|----------|
| 00 | What ROS2 is and why it matters |
| 01 | Installing ROS2, tools, and camera packages |
| 02 | Nodes, topics, services, actions, QoS — the core architecture |
| 03 | Workspace/package structure and *why* each file exists |
| 04 | Writing Python nodes with timers and parameters |
| 05 | Pub/Sub communication, message types, rosbag |
| 06 | Camera drivers, image transport, RViz visualization |
| 07 | cv_bridge + OpenCV: grayscale, edges, colour tracking |
| 08 | Deep learning: YOLO/MobileNet detection on live camera |

### Your Complete Pipeline

```
USB Camera  →  usb_cam driver  →  /camera/image_raw  →  YOLO Node  →  /detections/image
                                                                    →  /detections/json
```

From here, you can:
- Add a **planner node** that reads `/detections/json` and publishes `/cmd_vel`
- Integrate with **Nav2** for autonomous navigation
- Add **SLAM** for mapping
- Deploy to a **Jetson** or **Raspberry Pi**

**Happy building!** 🤖